In [ ]:
pip install geopandas rasterio


In [ ]:
import rasterio
import numpy as np


In [ ]:
input_path = "/content/lithology_30m_normalized.tif"

with rasterio.open(input_path) as src:
    data = src.read(1)  # Read the first band
    profile = src.profile.copy()  # Store metadata for later use
    nodata = src.nodata  # Get the NoData value if present


In [ ]:
# Create a mask for NoData values
if nodata is not None:
    mask = (data == nodata)
    data = np.where(mask, np.nan, data)  # Replace NoData with NaN for safe calculations
else:
    mask = np.isnan(data)  # Fallback if nodata is not set


In [ ]:
# Calculate min and max ignoring NaN
data_min = np.nanmin(data)
data_max = np.nanmax(data)

# Apply Min-Max normalization
data_norm = (data - data_min) / (data_max - data_min)
data_norm = np.clip(data_norm, 0, 1)


In [ ]:
nodata_replacement = 0  # You can change this if needed
data_norm[np.isnan(data_norm)] = nodata_replacement


In [ ]:
from rasterio.crs import CRS

profile.update({
    "dtype": "float32",
    "nodata": nodata_replacement,
    "crs": CRS.from_epsg(32643),  # WGS 84 / UTM Zone 43N
    "transform": src.transform,   # Keep original transform if already in same resolution
    "count": 1
})


In [ ]:
output_path = "normalized.tif"
profile.update(dtype='float32', nodata=nodata_replacement)

with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(data_norm.astype('float32'), 1)

print(f"Saved normalized raster to: {output_path}")


Saved normalized raster to: normalized.tif


In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling as ResampleMethod
from rasterio.crs import CRS
from rasterio.transform import from_origin
import numpy as np
import os

input_folder = "input_rasters"
output_folder = "normalized_rasters"
os.makedirs(output_folder, exist_ok=True)

target_crs = CRS.from_epsg(32643)  # WGS 84 / UTM Zone 43N
target_resolution = 30
nodata_replacement = 0

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".tif"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename.replace(".tif", "_normalized_30m.tif"))

        with rasterio.open(input_path) as src:
            # Compute bounds in target CRS
            dst_transform, dst_width, dst_height = rasterio.warp.calculate_default_transform(
                src.crs, target_crs, src.width, src.height, *src.bounds
            )

            # Force 30m resolution manually
            left, bottom, right, top = rasterio.warp.transform_bounds(
                src.crs, target_crs, *src.bounds
            )
            dst_width = int((right - left) / target_resolution)
            dst_height = int((top - bottom) / target_resolution)
            dst_transform = from_origin(left, top, target_resolution, target_resolution)

            # Prepare destination array
            destination = np.empty((dst_height, dst_width), dtype='float32')

            # Reproject with forced 30m resolution
            reproject(
                source=rasterio.band(src, 1),
                destination=destination,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dst_transform,
                dst_crs=target_crs,
                resampling=ResampleMethod.bilinear
            )

            # Normalize and handle NoData
            destination = np.where(destination == src.nodata, np.nan, destination)
            data_min = np.nanmin(destination)
            data_max = np.nanmax(destination)
            normalized = (destination - data_min) / (data_max - data_min)
            normalized = np.clip(normalized, 0, 1)
            normalized[np.isnan(normalized)] = nodata_replacement

            # Create output profile
            profile = src.profile.copy()
            profile.update({
                "crs": target_crs,
                "transform": dst_transform,
                "width": dst_width,
                "height": dst_height,
                "dtype": "float32",
                "nodata": nodata_replacement,
                "count": 1
            })

            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(normalized, 1)

            print(f"✔ Saved normalized 30m raster: {output_path}")


<ipython-input-16-6a9330dafc78>:54: RuntimeWarning: invalid value encountered in subtract
  normalized = (destination - data_min) / (data_max - data_min)
<ipython-input-16-6a9330dafc78>:54: RuntimeWarning: invalid value encountered in divide
  normalized = (destination - data_min) / (data_max - data_min)


✔ Saved normalized 30m raster: normalized_rasters/second_vertical_derivative_magnetic_2_normalized_30m.tif


<ipython-input-16-6a9330dafc78>:54: RuntimeWarning: invalid value encountered in divide
  normalized = (destination - data_min) / (data_max - data_min)


✔ Saved normalized 30m raster: normalized_rasters/analytic_signal_magnetic_2_normalized_30m.tif
✔ Saved normalized 30m raster: normalized_rasters/tilt_derivative_magnetic_2_normalized_30m.tif
